In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="1" 

import numpy as np
import tensorflow as tf
import pandas as pd
import pyarabic.araby as araby
import warnings
warnings.filterwarnings('ignore')
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.initializers import TruncatedNormal
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.metrics import CategoricalAccuracy
import torch
from sklearn.metrics import accuracy_score, f1_score
from transformers import Trainer, TrainingArguments
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from datasets import load_dataset, Dataset, concatenate_datasets
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', 1000)


fname = 'MSA_MDA_1'
log_file = fname + '.txt'

with open(log_file, 'w') as f:
    f.write('Model,Accuracy,F1\n')


df = pd.read_csv('datasets/MSA_MDA_comments.csv', encoding='utf-8', engine='python', sep='\t') #, quotechar="'"  , quoting=3

display(len(df))
df = df[df['language'] == 'mda']
      
display(df.columns)
display(df[:4])


display(len(df))

c = df['sentiment'].value_counts()
display(c)

classes = set(df['sentiment'].values)
display(classes)

df['sentiment'] = df['sentiment'].astype('category')
df['label'] = df['sentiment'].cat.codes

df = df[['text', 'label']]
classes_num = len(classes)
display(classes_num)
display(len(df))


max_sequence_length = 128



models = [ 
        'aubmindlab/bert-base-arabertv02-twitter',
        'CAMeL-Lab/bert-base-arabic-camelbert-da',
        'qarib/bert-base-qarib', 
        'UBC-NLP/MARBERT',
        'UBC-NLP/MARBERTv2',
    'SI2M-Lab/DarijaBERT-mix',
    'SI2M-Lab/DarijaBERT-arabizi'
]


seeds = [0, 1, 42]

for model_name in models:
    for seed in seeds:
        ds = Dataset.from_pandas(df)
        ds = ds.train_test_split(test_size=0.2, seed = seed)
        if seed==0:
            display(ds)
            
        for i in range(3):
            print(f'{model_name}, try:{i}')
                  
            tokenizer = AutoTokenizer.from_pretrained(model_name)
            model = AutoModelForSequenceClassification.from_pretrained(model_name,
                                                                  num_labels=classes_num).to('cuda')                                                 
            dataset_train = ds['train']
            dataset_validation = ds['test']                                                    
            
          
    
            def preprocess_function(examples):
                return tokenizer(examples['text'], truncation=True, padding="max_length",
                                max_length=max_sequence_length)
            
            
            dataset_train = dataset_train.map(preprocess_function, batched=True)
            dataset_validation = dataset_validation.map(preprocess_function, batched=True)
            
           
            
            def compute_metrics(eval_pred):
                logits, labels = eval_pred
                predictions = np.argmax(logits, axis=-1)    
                acc = accuracy_score(labels, predictions)        
                f1 = f1_score(labels, predictions, average='macro')   
                with open(log_file, 'a') as f:
                    f.write(f'{model_name},{acc},{f1}\n')
                return {'accuracy': acc, 'f1_score': f1}
    
    
            
            
            epochs = 15
            save_steps = 10000 #save checkpoint every 10000 steps
            batch_size = 64
            
            training_args = TrainingArguments(
                output_dir = 'bert/',
                overwrite_output_dir=True,
                num_train_epochs = epochs,
                per_device_train_batch_size = batch_size,
                per_device_eval_batch_size = batch_size,
                save_steps = save_steps,
                save_total_limit = 1, #only save the last 5 checkpoints
                fp16=True,
                learning_rate = 5e-5,  # 5e-5 is the default
                logging_steps = 25, #50_000
                evaluation_strategy = 'steps',
                # evaluate_during_training = True,
                eval_steps = 25
                
            )
            
            trainer = Trainer(
                model = model,
                args = training_args,
                # data_collator=data_collator,
                train_dataset=dataset_train,
                eval_dataset=dataset_validation,
                compute_metrics = compute_metrics
            )
            
            
            trainer.train()


results = pd.read_csv(log_file)

best_results = results.groupby('Model', as_index=False)['F1'].max()

best_results = pd.merge(best_results, results, on=['Model', 'F1'])
best_results = best_results[['Model', 'Accuracy', 'F1']]
best_results = best_results.drop_duplicates()
best_results.to_csv(f'{fname}.csv')
display(best_results)



2024-09-18 17:19:36.631510: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-09-18 17:19:36.654432: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI AVX512_BF16 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-09-18 17:19:37.031362: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


9901

Index(['text', 'language', 'sentiment'], dtype='object')

,text,language,sentiment
6359,اجي هاك الطاجين راك فزت به سربي ما حدو سخون. بالصحة والعافية.,mda,positive
6360,اخصنى انديروها ابنات ميدالت فالصيف فالمو صمورة متافقين ويشجعون ولداتنا تخيلوها هههههههه,mda,positive
6361,الفلاح هوا العمود الفقري ديال المغرب تبرك الله عليك الله يعونك,mda,positive
6362,كاين الخير والأرض خضارت والفلاح نشط مع هاد الأمطار غير كاين سمر والبرد مع الصباح حيت ليالي قريبة والحمد القوي العزيز,mda,positive


3542

...

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at SI2M-Lab/DarijaBERT-arabizi and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/2833 [00:00<?, ? examples/s]

Map:   0%|          | 0/709 [00:00<?, ? examples/s]

Step,Training Loss,Validation Loss,Accuracy,F1 Score
25,0.371200,0.283534,0.894217,0.800458
50,0.257000,0.278784,0.892807,0.823132
75,0.147800,0.275605,0.905501,0.839823
100,0.126400,0.593856,0.816643,0.762301
125,0.121400,0.399441,0.912553,0.834441
150,0.081600,0.361694,0.901269,0.838968
175,0.037300,0.397626,0.904090,0.835920
200,0.009600,0.624408,0.897038,0.828606
225,0.024300,0.580318,0.898449,0.831454
250,0.001800,0.603306,0.901269,0.835160


,Model,Accuracy,F1
0,CAMeL-Lab/bert-base-arabic-camelbert-da,0.922426,0.871628
3,SI2M-Lab/DarijaBERT-arabizi,0.915374,0.859545
6,SI2M-Lab/DarijaBERT-mix,0.936530,0.900671
9,UBC-NLP/MARBERT,0.926657,0.875301
12,UBC-NLP/MARBERTv2,0.937941,0.898196
15,aubmindlab/bert-base-arabertv02-twitter,0.939351,0.896565
21,qarib/bert-base-qarib,0.936530,0.893072
